In [ ]:
!python3 -m pip install pandas dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 62.8 MB/s eta 0:00:00


In [ ]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

--2024-08-10 15:05:05--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2476 (2.4K) [text/csv]
Saving to: ‘spacex_launch_dash.csv’

spacex_launch_dash. 100%[===================>]   2.42K  --.-KB/s    in 0s      

2024-08-10 15:05:06 (565 MB/s) - ‘spacex_launch_dash.csv’ saved [2476/2476]



In [ ]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py"

--2024-08-10 15:05:13--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2110 (2.1K) [text/x-python]
Saving to: ‘spacex_dash_app.py’

spacex_dash_app.py  100%[===================>]   2.06K  --.-KB/s    in 0s      

2024-08-10 15:05:14 (993 MB/s) - ‘spacex_dash_app.py’ saved [2110/2110]



In [ ]:
!python3 spacex_dash_app.py

/content/spacex_dash_app.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/content/spacex_dash_app.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app 'spacex_dash_app'
 * Debug mode: off
 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
^C


In [ ]:
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [ ]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Dropdown list(s)
launch_site_list = []
launch_site_list.append('ALL')
for index, row in spacex_df['Launch Site'].value_counts().to_frame().iterrows():
    launch_site_list.append(row.name)

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                    options=[{'label': i, 'value': i} for i in launch_site_list],
                                    style={'width':'100%', 'padding':'3px', 'font-size': '20px', 'text-align-last': 'left'},
                                    value='ALL'),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider', min=min_payload, max=max_payload, step=1000, value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        filtered_df = spacex_df[['Launch Site', 'class']].groupby(by=['Launch Site'], as_index=False).mean()
        fig = px.pie(filtered_df, values='class',
                     names='Launch Site',
                     title='Total Success Launches by Site')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df = spacex_df[['Launch Site', 'class']][spacex_df['Launch Site'] == entered_site]
        mean = filtered_df.groupby(by='Launch Site', as_index=False).mean()
        means = {}
        means[1] = mean['class'][0]
        means[0] = 1 - means[1]
        fig = px.pie(values=means.values(), names=means.keys(),
                     title=f'Total Success Launches by Site: {entered_site}')
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id="payload-slider", component_property="value")])

def get_scatter_plot(entered_site, payload_range):
    # print('min:', payload_range[0], '\tmax:', payload_range[1])
    # print(entered_site)
    if entered_site == 'ALL':
        payload_filtered_df = spacex_df[['Payload Mass (kg)', 'Booster Version Category', 'Launch Site', 'class']][(spacex_df['Payload Mass (kg)'] <= payload_range[1]) & (spacex_df['Payload Mass (kg)'] >= payload_range[0])]
    else:
        payload_filtered_df = spacex_df[['Payload Mass (kg)', 'Booster Version Category', 'Launch Site', 'class']][(spacex_df['Payload Mass (kg)'] <= payload_range[1]) &
                                                                                                                   (spacex_df['Payload Mass (kg)'] >= payload_range[0]) &
                                                                                                                   (spacex_df['Launch Site'] == entered_site)]
    fig = px.scatter(data_frame=payload_filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category')
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


# Finding Insights Visually
# Now with the dashboard completed, you should be able to use it to analyze SpaceX launch data, and answer the following questions:
#
# Which site has the largest successful launches?
### KSC LC-39A
# Which site has the highest launch success rate?
### KSC LC-39A
# Which payload range(s) has the highest launch success rate?
### 2000 - 4000
# Which payload range(s) has the lowest launch success rate?
### 6000 - 9000
# Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest launch success rate?
### B5

<IPython.core.display.Javascript object>

In [ ]:
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

launch_sites = []
launch_sites.append({'label': 'All Sites', 'value': 'All Sites'})
all_launch_sites = spacex_df['Launch Site'].unique().tolist()
for launch_site in all_launch_sites:
    launch_sites.append({'label': launch_site, 'value': launch_site})

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                html.Div([
                                    dcc.Dropdown(
                                        id = 'site-dropdown',
                                        options = launch_sites,
                                        placeholder = 'Select a Launch Site here',
                                        searchable = True ,
                                        clearable = False,
                                        value = 'All Sites'
                                        ),]),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                html.Div([
                                    dcc.RangeSlider(
                                        id = 'payload_slider',
                                        min = 0,
                                        max = 10000,
                                        step = 1000,
                                        marks = {
                                            0: {'label': '0 Kg', 'style': {'color': '#77b0b1'}},
                                            1000: {'label': '1000 Kg'},
                                            2000: {'label': '2000 Kg'},
                                            3000: {'label': '3000 Kg'},
                                            4000: {'label': '4000 Kg'},
                                            5000: {'label': '5000 Kg'},
                                            6000: {'label': '6000 Kg'},
                                            7000: {'label': '7000 Kg'},
                                            8000: {'label': '8000 Kg'},
                                            9000: {'label': '9000 Kg'},
                                            10000: {'label': '10000 Kg', 'style': {'color': '#f50'}},
                                            },
                                            value = [min_payload,max_payload]),
                                            ]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
     Output(component_id = 'success-pie-chart', component_property = 'figure'),
     [Input(component_id = 'site-dropdown', component_property = 'value')]
)
def update_piegraph(site_dropdown):
    if (site_dropdown == 'All Sites' or site_dropdown == 'None'):
        data  = spacex_df[spacex_df['class'] == 1]
        fig = px.pie(
                data,
                names = 'Launch Site',
                title = 'Total Success Launches by All Sites',
            )
    else:
        data  = spacex_df.loc[spacex_df['Launch Site'] == site_dropdown]
        fig = px.pie(
                data,
                names = 'class',
                title = 'Total Success Launches for Site &#8608; '+site_dropdown,
            )
    return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
     Output(component_id = 'success-payload-scatter-chart', component_property = 'figure'),
     [Input(component_id = 'site-dropdown', component_property = 'value'),
     Input(component_id = "payload_slider", component_property = "value")]
)
def update_scattergraph(site_dropdown,payload_slider):
    if (site_dropdown == 'All Sites' or site_dropdown == 'None'):
        low, high = payload_slider
        data  = spacex_df
        inrange = (data['Payload Mass (kg)'] > low) & (data['Payload Mass (kg)'] < high)
        fig = px.scatter(
                data[inrange],
                x = "Payload Mass (kg)",
                y = "class",
                title = 'Correlation Between Payload and Success for All Sites',
                color="Booster Version Category",
                size='Payload Mass (kg)',
                hover_data=['Payload Mass (kg)']
            )
    else:
        low, high = payload_slider
        data  = spacex_df.loc[spacex_df['Launch Site'] == site_dropdown]
        inrange = (data['Payload Mass (kg)'] > low) & (data['Payload Mass (kg)'] < high)
        fig = px.scatter(
                data[inrange],
                x = "Payload Mass (kg)",
                y = "class",
                title = 'Correlation Between Payload and Success for Site &#8608; '+site_dropdown,
                color="Booster Version Category",
                size='Payload Mass (kg)',
                hover_data=['Payload Mass (kg)']
            )
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server()

<IPython.core.display.Javascript object>